# Unwetter Simulator

In [ ]:
import os
os.chdir('..')


f'Working directory: {os.getcwd()}'

In [ ]:
from unwetter import db, map
from datetime import datetime

In [ ]:
from unwetter import config

config.SEVERITY_FILTER = ['Severe', 'Extreme']
config.STATES_FILTER = ['NW']
config.EMERGENCY_FILTER = ['Immediate']

In [ ]:
search_start = datetime(2019, 6, 19, 11, 0)
search_end = datetime(2019, 6, 19, 22, 0)

search_filter = {
    '$and': [
        {
            'sent': {
                '$gt': search_start,
            },
        },
        {
            'sent': {
                '$lt': search_end,
            },
        },
    ]
}

events = list(db.collection.find(search_filter))

In [ ]:
len(events)

In [ ]:
len([e for e in events if e['published']])

In [ ]:
for e in events:
    e['published'] = False

In [ ]:
def mock_by_ids(ids):
    return [event for event in events if event['id'] in ids]

def mock_publish(ids):
    for event in events:
        if event['id'] in ids:
            event['published'] = True

In [ ]:
from unwetter.config import filter_event

def mock_update(new_events):
    filtered = []
    for event in new_events:
        if filter_event(event):
            if event['msg_type'] in ['Alert', 'Cancel']:
                filtered.append(event)

            elif not any(t['changed'] for t in event['has_changes']):
                continue

            elif any(t['changed'] for t in event['has_changes']):
                filtered.append(event)

            else:
                print(f'Event was not filtered 1: {event["id"]}')

        else:
            if event['msg_type'] in ['Alert', 'Cancel']:
                continue

            else:
                old_events = mock_by_ids(event['references'])

                if any((old_event['published'] and filter_event(old_event)) for old_event in old_events):
                    filtered.append(event)

                elif not any(old_event['published'] for old_event in old_events):
                    continue

                else:
                    print(f'Event was not filtered 2: {event["id"]}')

    mock_publish([event['id'] for event in filtered])
    return filtered

In [ ]:
current_sent = events[0]['sent']

bins = []
current_bin = []

for event in events:
    if event['sent'] != current_sent:
        current_sent = event['sent']
        bins.append(current_bin)
        current_bin = []
    
    current_bin.append(event)

bins.append(current_bin)


In [ ]:
sum(len(mock_update(bin)) for bin in bins)